In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import urllib

/home/sajid/miniconda3/envs/ipy3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ZP = np.load("zp.npy")

In [3]:
def get_property(mat,energy):
    url = "http://henke.lbl.gov/cgi-bin/pert_cgi.pl"
    data = {'Element':str(mat), 'Energy':str(energy), 'submit':'Submit Query'}
    data = urllib.parse.urlencode(data)
    data = data.encode('utf-8')
    req = urllib.request.Request(url, data)
    resp = urllib.request.urlopen(req)
    respDat = resp.read()
    response = respDat.split()
    d =  b'g/cm^3<li>Delta'
    i = response.index(d)
    delta = str(response[i+2])[:str(response[i+2]).index('<li>Beta')][2:]
    beta = str(response[i+4])[2:-1]
    return float(delta),float(beta)

In [6]:
energy = 25000                     
delta,beta = get_property('Au',energy)
wavel = (1239.84/energy)*10**(-9)  
k = 2*np.pi/wavel

dim_x = 32768
dim_y = 32768
dim_xy = dim_x*dim_y    

In [7]:
ZP = ZP.reshape(dim_xy)
ZP_ = np.zeros((dim_xy,2))
ZP_[:,0] = np.real(ZP)
ZP_[:,1] = np.imag(ZP)

In [8]:
f = h5py.File("ref_index.h5", "w")
dset = f.create_dataset("ref_index", np.shape(ZP_), dtype='f8')
dset.attrs.__setitem__("complex",1)

In [9]:
dset[:,:] = ZP_

In [10]:
f.close()